In [89]:
import pandas as pd
import numpy as np
from datetime import datetime
import vaex

## remarks:
### currently no fill null strategy, no function for converting string to datetime, no two datetime direct subtraction, must fill all the nulls before do aggregation
### Therefore need to do it in pandas first

In [20]:
n_rows = 1000000
n_cols = 500
pandas_df = pd.DataFrame(np.random.randint(0, 100, size=(n_rows, n_cols)), columns=['col%d' % i for i in range(n_cols)])
pandas_df.to_csv('final_data.csv', index=False)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
0,71,45,68,82,84,32,56,78,62,55,...,97,41,93,57,9,70,77,62,69,21
1,24,5,70,48,8,19,82,7,17,18,...,42,34,2,54,40,68,36,9,89,41
2,54,64,38,70,64,73,80,97,9,10,...,30,44,73,14,63,99,84,17,49,92
3,86,54,59,9,87,23,73,88,69,15,...,28,74,38,22,58,84,86,48,76,47
4,68,29,71,57,61,27,25,3,25,69,...,10,16,96,65,7,82,11,9,22,39


In [15]:
# this read the dataframe and make a copy of the file in hdf5 format 
df = vaex.from_csv('final_data.csv', convert=True, chunk_size=5_000_000) 

In [141]:
student = {'name': ['Tom Chan', 'Sam Cheung', 'Amy Chan', 'John Lee', 'Ben Wong', 'May Chan'], 
           'class': ['1A', '1A', '1B', '1B', '1C', '1C'], 
           'age':[12, 15, 12, 17, 7, 20], 
           'sex': ['M', 'M', 'F', 'M', 'M', 'F'],
           'birthday':[datetime(1999,8,12), datetime(1999,1,12), datetime(1999,10,12), datetime(2000,8,20), datetime(2001,7,3), datetime(1999,11,26)], 
           'score': [76, 59, 79, 60, 71, 90], 
           'intro':['He is a good boy', 'He is a bad boy', 'She has long hair', 'He love maths', 'He feels happy', 'She is tall']}

In [142]:
pandas_df = pd.DataFrame(student)

In [143]:
df = vaex.from_pandas(pandas_df)

In [144]:
df.describe(strings=False)

,age,birthday,score
data_type,int64,datetime64[ns],int64
count,6,6,6
NA,0,0,0
mean,13.833333333333334,2000-01-29T00:00:00.000000000,72.5
std,4.139914,25037571153769892.0,10.812801
min,7,1999-01-12T00:00:06.928662528,59
max,20,2001-07-03T00:00:21.731868672,90


In [145]:
df.rename('score', 'marks')

'marks'

In [146]:
df['class'].value_counts()

1C    2
1B    2
1A    2
dtype: int64

In [127]:
df['grading'] = df.func.where(df['marks'] >= 70, 'Pass', 'Fail')

In [128]:
df[df['class']=='1A'].sum('marks')

array(135, dtype=int64)

In [130]:
df['year'] = df['birthday'].dt.year

In [138]:
df.groupby('sex', agg={'marks':'sum'})

#,sex,marks
0,M,266
1,F,169


In [159]:
df.count(binby=df['age'], limits=[0, 100], shape=10)

array([1, 4, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [150]:
df['intro'].str.len()

Expression = str_len(intro)
Length: 6 dtype: int64 (expression)
-----------------------------------
0  16
1  15
2  17
3  13
4  14
5  11